In [ ]:
import re

stan_simple = """
data {
    int n;
    vector[n] y;
    vector<lower=0> sigma;
}
parameters {
    vector[n] theta;
    real mu;
    real<lower=0> tau;
}
model {
    y ~ normal(theta, sigma);
    theta ~ normal(mu, tau);
}
"""

stan_with_tp = """
data {
    int n;
    vector[n] y;
    vector<lower=0> sigma;
}
parameters {
    vector[n] theta;
    real mu;
    real<lower=0> tau;
}
transformed parameters {
    real a = tau * 2;
}
model {
    y ~ normal(theta, sigma);
    theta ~ normal(mu, tau);
}
"""

In [ ]:
def get_parameter_names_and_types(stan_code):
    # find the names of parameters in a stan program
    param_block = re.search(r"parameters\s+{([\S\s]*?)\}", stan_code).groups()[0]
    return {
        line.split()[-1].strip(";"): line.split()[0].strip()
        for line in param_block.splitlines()[1:]
    }

def get_parameter_dists(parameter_names, stan_code):
    # find the distribution of parameters in a stan program
    model_block = re.search(r"model\s+{([\S\s]*?)\}", stan_code).groups()[0]
    param_to_dist = {}
    for line in model_block.splitlines()[1:]:
        param, dist = line.strip().split("~")
        param = param.strip()
        dist = dist.strip(";").strip()
        if param in parameter_names:
            param_to_dist[param] = dist
    return param_to_dist

def is_normal(dist):
    return ("normal" in dist) and ("lognormal" not in dist)

def get_mu_sigma(dist):
    mu, sigma = re.search(r"normal\((.+)\,\s+(.+)\)", dist).groups()
    return mu, sigma

def modify_transformed_parameters_block(name_to_type, param_to_dist, stan_code):
    # create a new transformed parameters block
    try:
        transformed_param_block = re.search(
            r"(transformed parameters\s+{[\S\s]*?\})", 
            stan_code
        ).groups()[0]
    except AttributeError:
        transformed_param_block = None
        
    transformed_params_lines = []
    for param_name, dist in param_to_dist.items():
        mu, sigma = get_mu_sigma(dist)
        transformed_param_line = ("  " 
                                  + name_to_type[param_name] 
                                  + " " + param_name 
                                  + " = " 
                                  + mu 
                                  + " + " 
                                  + param_name 
                                  + "_std * " 
                                  + sigma 
                                  + ";\n")
        transformed_params_lines.append(transformed_param_line)
        
    if transformed_param_block is None:
        return "transformed parameters {\n" + "\n".join(transformed_params_lines) + "}\n"
    else:
        original_lines = transformed_param_block.splitlines()
        new_lines = (
            [original_lines[0]] 
            + transformed_params_lines 
            + original_lines[1:-1]
            + ["}"]
        )
        return "\n".join(new_lines)
    
    
def modify_parameters_block(param_names, stan_code):
    param_block = re.search(r"(?<!transformed )(parameters\s+{[\S\s]*?\})", stan_code).groups()[0]
    for param in param_names:
        param_block = param_block.replace(param, param + "_std")
    return param_block


def modify_model_block(param_names, stan_code):
    model_block = re.search(r"(model\s+{[\S\s]*?\})", stan_code).groups()[0]
    new_lines = []
    for line in model_block.splitlines():
        found_param = False
        for param in param_names:
            if param in line.split("~")[0]:
                non_cent_line = "  " + param + "_std ~ normal(0, 1);"
                new_lines.append(non_cent_line)
                found_param = True
        if not found_param:
            new_lines.append(line)
    return "\n".join(new_lines)
    
    
def make_non_centered(stan_code):
    name_to_type = get_parameter_names_and_types(stan_code)
    param_to_dist = get_parameter_dists(name_to_type.keys(), stan_code)
    norm_params = {param_name: dist for param_name, dist in param_to_dist.items() if is_normal(dist)}
    std_normal_params = [param_name + "_std" for param_name in norm_params.keys()]
    
    transformed_params_block = modify_transformed_parameters_block(name_to_type, norm_params, stan_code)
    param_block = modify_parameters_block(norm_params.keys(), stan_code)
    model_block = modify_model_block(norm_params.keys(), stan_code)
    
    if re.search(r"(transformed parameters\s+{[\S\s]*?\})", stan_code) is None:
        new_stan_code = re.sub(
            r"model\s+{([\S\s]*?)\}", 
            transformed_params_block + model_block, 
            stan_code
        )
    else:
        new_stan_code = re.sub(r"model\s+{([\S\s]*?)\}", model_block, stan_code)
        new_stan_code = re.sub(r"(transformed parameters\s+{[\S\s]*?\})", transformed_params_block, new_stan_code)
    
    new_stan_code = re.sub(r"(?<!transformed )(parameters\s+{[\S\s]*?\})", param_block, new_stan_code)
    
    
    return new_stan_code

In [ ]:
print(make_non_centered(stan_with_tp))

In [ ]:
schools_code = """
data {
    int<lower=0> J; // number of schools
    vector[J] y; // estimated treatment effects
    vector<lower=0>[J] sigma; // s.e. of effect estimates
}
parameters {
    real mu;
    real<lower=0> tau;
    vector[J] theta;
}
model {
    theta ~ normal(mu, tau);
    y ~ normal(theta, sigma);
}
"""

In [ ]:
print(make_non_centered(schools_code))

In [ ]:
x = """

data {
    int<lower=1> nteam;
    int<lower=1> nmatch;
    int home_team[nmatch];
    int away_team[nmatch];
    int home_goals[nmatch];
    int away_goals[nmatch];
}
parameters {
    vector[nteam] log_a;
    vector[nteam] log_b;
    real<lower=0> gamma;
    real<lower=0> sigma_a;
    real<lower=0> sigma_b;
    real mu_b;
}
transformed parameters {
    vector[nteam] a = exp(log_a);
    vector[nteam] b = exp(log_b);
    vector[nmatch] home_rate = a[home_team] .* b[away_team] * gamma;
    vector[nmatch] away_rate = a[away_team] .* b[home_team];
}
model {
    gamma ~ lognormal(0, 1);
    sigma_a ~ normal(0, 1);
    sigma_b ~ normal(0, 1);
    mu_b ~ normal(0, 1);
    log_a ~ normal(0, sigma_a);
    log_b ~ normal(mu_b, sigma_b);
    home_goals ~ poisson(home_rate);
    away_goals ~ poisson(away_rate);
}
"""

In [ ]:
y = make_non_centered(x)

In [ ]:
print(y)